In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.12.1-DEV ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


In [2]:
cd("..")

In [107]:
currentDir = pwd() # make sure you are running from the main directory.
include(joinpath(currentDir, "src/fileHandling.jl"))
include(joinpath(currentDir, "src/isolate3Lines.jl"))
include(joinpath(currentDir, "src/matroid_realization.jl"))
include(joinpath(currentDir, "src/Jacobian_Criterion.jl"))

realization_space_2_singular_locus (generic function with 1 method)

First we isolate those that satisfy the three lines property, i.e., every element of the ground set is contained in at least 3 lines (cyclic flats of rank 2). 

In [4]:
db = Polymake.Polydb.get_db();
collection = db["Matroids.Small"];
d3n10 = Polymake.Polydb.find(collection, Dict("RANK" => 3, "SIMPLE"=>true, "N_ELEMENTS"=>10));


n3C10 = subsets(collect(1:10), 3);
n3C10 = sort(n3C10, by =  x-> reverse(x));

lines_3 = []
for c in d3n10
    Q = Matroid(c)
    ns = count_3_lines_thru_all_points(Q)
    if length(ns) == 0
        continue
    end    
    if minimum(ns) >= 3
        push!(lines_3, to_revlex(Q, n3C10))
    end
end

This data is precomputed in `d3n10/3lines_d3n10.dat`.

In [5]:
lines_3_precomputed = vec(readlines("d3n10/3lines_d3n10.dat"))

151-element Vector{String}:
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "***************0**0****0*******"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "***************00**********0***"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "**************0******0*0*******"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "**************0******0***0*****"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "**********0*******0*****0******"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "**********0********0****0******"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "**********0**********0****0****"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "***********0****0*********0****"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "*******0**************00*******"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "*****0****************0***0****"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "*****0****************0*0******"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "*****0*************0*

In [ ]:
Set(lines_3) == Set(lines_3_precomputed)

The file `d3n10/not_realizable.dat` contains those matroids in `d3n10/3lines_d3n10.dat` that are not realizable over $\mathbb{Q}$. 

In [6]:
not_realizable = vec(readlines("d3n10/nonrealizable.dat"));

In [ ]:
not_realizable = vec(readlines("d3n10/nonrealizable.dat"))

test_nonrealizable = []

for Qstr in not_realizable
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    As = rank_plus1_circuits(Q) 
    MR = new_matroid_realization_space(Q, As[1]; F=QQ, saturate=true)
    push!(test_nonrealizable, !MR.representable)
end

In [ ]:
all(test_nonrealizable)

The files `d3n10/zero_ideal.dat`, `d3n10/univariate_ideal.dat`, and `d3n10/principal_ideal.dat` record matroids and a maximal circuit. A pair of matroid and circuit is recorded as a `*/0` vector. The first 10 entries record the circuit (e.g., `**0*00*000` is the circuit `[1,2,4,7]`) and the rest record the matroid in the usual revlex basis encoding. 

The files `d3n10/zero_ideal.dat` contains those matroids in `d3n10/3lines_d3n10.dat` and a maximal circuit such that the ideal of the realization space is zero. Therefore these realization spaces are smooth. 

In [97]:
zero_ideal = vec(readlines("d3n10/zero_ideal.dat"))

test_zero = []
for cir_Qstr in zero_ideal
    A = [i for i in 1:10 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[11:130]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    push!(test_zero, iszero(I))
end

In [98]:
all(test_zero)

true

The files `d3n10/univariate_ideal.dat` contains those matroids in `d3n10/3lines_d3n10.dat` and a maximal circuit such that the ideal of the realization space is univariate. These realization spaces are smooth provided the discriminant is nonzero. 

In [102]:
univariate_ideal = vec(readlines("d3n10/univariate_ideal.dat"))

test_univariate = []
for cir_Qstr in univariate_ideal
    A = [i for i in 1:10 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[11:130]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    length_vs = length(ideal_vars(gens(I))) 
    push!(test_univariate, isone(length_vs))
end

In [103]:
all(test_univariate)

true

The files `d3n10/principal_ideal.dat` contains those matroids in `d3n10/3lines_d3n10.dat` and a maximal circuit such that the ideal of the realization space is univariate. One could check that these are smooth using the `singular_locus` function in `src/Jacobian_Criterion.jl`, but we do this explicitly here. 

In [113]:
principal_ideal = vec(readlines("d3n10/univariate_ideal.dat"))

test_principal = []
for cir_Qstr in univariate_ideal
    A = [i for i in 1:10 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[11:130]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    R = MR.ambient_ring
    x = gens(R)
    I = MR.defining_ideal
    Igens = gens(I)
    length(Igens) != 1 && error("not principal") 
    JM = jacobian_matrix(R, x, Igens)
    nr, nc = size(JM) 
    J = I + ideal(R, [JM[1,c] for c in 1:nc])
    Sing = stepwise_saturation(J, MR.inequations)
    #Sing = realization_space_2_singular_locus(MR)
    push!(test_principal, isone(Sing))
end

In [115]:
all(test_principal)

true

### Scratch (to remove)

In [8]:
## add this to where rank_plus1_circuits is

function count_nonbases_meeting_circuit_2elem(Q, A)
    NBs = nonbases(Q)
    return length([nb for nb in NBs if length(intersect(A,nb)) == 2])
end


function optimal_circuits(Q)
    As = rank_plus1_circuits(Q)
    mx = maximum([count_nonbases_meeting_circuit_2elem(Q, A) for A in As])
    return [A for A in As if count_nonbases_meeting_circuit_2elem(Q, A) == mx] 
end

optimal_circuits (generic function with 1 method)

In [9]:


remaining = setdiff(lines_3_precomputed, not_realizable)

zero = []
univariate = []
principal = []
not_principal = []
new_nonrealizable = []
for Qstr in remaining
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    As = optimal_circuits(Q)
    A = As[1]
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
    
    if !MR.representable
        push!(new_nonrealizable, Qstr)
        continue
    end
    
    Qstr = to_star0(A,10) * Qstr
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    if iszero(I)
        push!(zero, Qstr)
    elseif length(ideal_vars(gens(I))) == 1
        push!(univariate, Qstr)
    elseif length(gens(I)) == 1
        push!(principal, Qstr)
    else
        push!(not_principal, Qstr)
    end 
    
end

In [40]:
cantoptimizefurther = []

Any[]

In [61]:
principal12 = []
for Qstr in cantoptimizefurther
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    As = optimal_circuits(Q) 
    A = As[1]
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
    
    if !MR.representable
        push!(new_nonrealizable, Qstr)
        continue
    end
    
    Qstr = to_star0(A,10) * Qstr
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    if iszero(I)
        push!(zero, Qstr)
    elseif length(ideal_vars(gens(I))) == 1
        push!(univariate, Qstr)
    elseif length(gens(I)) == 1
        push!(principal12, Qstr)
    else
        push!(not_principal, Qstr)
    end 
    
end

In [83]:
principal = []
for Qstr in cantoptimizefurther
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    As = optimal_circuits(Q) 
    A = As[1]
    Qstr = to_star0(A,10) * Qstr
    push!(principal, Qstr)    
end

In [69]:
done = vcat(zero, univariate)
unique!(done)
done2 = [a[11:130] for a in done]

79-element Vector{String}:
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "**********0*******0*****0******"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "*****0****************0*0******"
 "0******0******0***0************" ⋯ 58 bytes ⋯ "**************0******0***0*****"
 "0******0******0***0************" ⋯ 58 bytes ⋯ "************0********0*****0***"
 "0******0******0***0************" ⋯ 58 bytes ⋯ "**************0******0****0****"
 "0******0******0***0************" ⋯ 58 bytes ⋯ "*************0*******0*****0***"
 "0******0******0***0************" ⋯ 58 bytes ⋯ "*************0******0*******0**"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "***************00*******0******"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "***************0***0****0******"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "***************00**********0***"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "***********0*********0*****0***"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "**********0*******0***

In [ ]:
nr = vcat(not_realizable, new_nonrealizable)
sort!(nr)
io = open("d3n10/new_nonrealizable.dat", "w")
for a in nr
    write(io, a, "\n")
end
close(io)

In [85]:
unique!(principal);
principal2 = [a[11:130] for a in principal]
unique!(principal2)
length(principal2)
 # == 
#length(principal)

26

In [86]:
io = open("d3n10/principal_ideal.dat", "w")
for a in principal
    write(io, a, "\n")
end
close(io)